# Crud Ops with `mongoengine`

In [1]:
from mongoengine import Document, StringField, EmailField, IntField, ValidationError, connect, disconnect
from pymongo import MongoClient

disconnect()
db = connect(host='mongodb://mongodb:27017/mongoengine')
db.drop_database('mongoengine')


class User(Document):
    username = StringField(required=True, unique=True, min_length=4, max_length=50)
    email = EmailField(required=True, unique=True)
    age = IntField(min_value=18, max_value=100)

    def clean(self):
        # Custom validation
        if not self.username.isalnum():
            raise ValidationError("Username should contain only alphanumeric characters")


# Ensure indexes are created
User.ensure_indexes()

## Create

In [3]:
user = User.objects.create(username="Alice", email="alice@example.net", age=18)

In [4]:
user = User.objects.filter(username="Alice").first()
user.to_json()

'{"_id": {"$oid": "66670c5f9bc009480646e9ed"}, "username": "Alice", "email": "alice@example.net", "age": 18}'

## Update

In [5]:
user.email = "alice@ordergoove.com"
user.save()

User.objects.filter(username="Alice").update(age=28)

1

In [6]:
user = User.objects.filter(username="Alice")
user.to_json()

'[{"_id": {"$oid": "66670c5f9bc009480646e9ed"}, "username": "Alice", "email": "alice@ordergoove.com", "age": 28}]'

## Relationships between models

In [7]:
from mongoengine import ReferenceField


# Define the Post model --- CONSTRAINTS ARE ENFORCED BY MONGOENGINE ITSELF, NOT BY PYMONGO
class Post(Document):
    title = StringField(required=True, min_length=1, max_length=200)
    content = StringField(required=True, min_length=1)
    author = ReferenceField(User, required=True)


# Ensure indexes are created
Post.ensure_indexes()

In [8]:
author = User.objects.get(username="Alice")

post = Post(title="First Post", content="This is the content of the first post.", author=author)
post.save()

<Post: Post object>

In [9]:
post = Post.objects.first()
post.to_json()

'{"_id": {"$oid": "66670c8c9bc009480646e9ee"}, "title": "First Post", "content": "This is the content of the first post.", "author": {"$oid": "66670c5f9bc009480646e9ed"}}'

In [10]:
post.author, post.author.to_json()

(<User: User object>,
 '{"_id": {"$oid": "66670c5f9bc009480646e9ed"}, "username": "Alice", "email": "alice@ordergoove.com", "age": 28}')

## Delete

In [11]:
post.delete()
user.delete()

1

In [12]:
Post.objects.all()

[]

In [13]:
User.objects.all()

[]

# PyMongo